<a href="https://colab.research.google.com/github/gladwel/DIP/blob/main/DAO_summarization_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis

Exploratory data analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. This step is absolutely necessary and has a huge impact on the final result of a model, this analysis will tell us what type of transformation we need to apply to our data. When we face a machine learning problem, we must begin by analyzing the input dataset, seeking to identify its characteristics and anomalies.

In [ ]:
data_path = 'data/'
data_filename='Inshorts Cleaned Data.xlsx'
output_path='data_analysis/'
# Load the data from the file system
summary = pd.read_excel(data_path + data_filename)
summary.head(5)
# Drop unecessary columns
summary.drop(labels=['Source ','Time ', 'Publish Date'], axis=1)
#Remove duplicates on the ctext 
summary.drop_duplicates(subset=["Short"],inplace=True)
#Remove rows containing null values
summary.dropna(inplace=True)
#Recreate the dataframe index
summary.reset_index(drop=True,inplace=True)
summary.describe()